<a href="https://colab.research.google.com/github/whoiswentz/tcc-neural-network/blob/master/densenet_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.applications.densenet import DenseNet169, DenseNet201
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model

from tensorflow.keras.models import load_model

from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras import activations


In [3]:
# Mount the Google Drive folder to get the dataset
from google.colab import drive

GOOGLE_DRIVE_MOUNT_DIR = "/content/drive"
drive.mount(GOOGLE_DRIVE_MOUNT_DIR, force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def create_densenet_model(output_space):
    # Get back the convolutional part of a Densenet network trained on ImageNet
    base_densenet_model = DenseNet169(weights='imagenet', include_top=False, input_shape=(175, 115, 3))

    #Add a layer where input is the output of the second last layer 
    x = Flatten(name="flatten")(base_densenet_model.layers[-2].output)
    x = Dense(output_space, activation=activations.softmax, name='predictions')(x)

    #Create your own model
    densenet = Model(inputs=base_densenet_model.get_input_at(0), outputs=x)
    
    # Freezing the first five layers
    #half_number_of_layers = int(np.floor(len(densenet.layers) - 2))
    #for layer in densenet.layers[:half_number_of_layers]:
    #    layer.trainable = False

    # Compoling the Densenet model
    # adam
    # rmsprop
    densenet.compile(loss="categorical_crossentropy", optimizer="adam")
    
    # Show the neural network summary
    # densenet.summary()

    return densenet

# Tumor class

The class of breast tumors are classified in two classes, benign  and malignant

In [0]:
TRAIN_PATH = "../content/drive/My Drive/BreakHis/Dataset/train/Tumor Class"
CHECK_POINT_PATH = "../content/drive/My Drive/TCC Models/Tumor Class/{}.hdf5"

In [0]:
BATCH_SIZE = 64
EPOCHS = 50

# 22 + 23 
#densenet_tumor_class = create_densenet_model(2)

model_file = "tumor_class_5_data_aug-21-0.44"
model_path = CHECK_POINT_PATH.format(model_file)
densenet_tumor_class = load_model(model_path)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
TARGET_SIZE = (175, 115)

datagen = ImageDataGenerator(
    horizontal_flip=True, 
    vertical_flip=True,
    validation_split=0.25)

train_data_generator = datagen.flow_from_directory(
    TRAIN_PATH,
    shuffle=True,
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training")

validation_data_generator = datagen.flow_from_directory(
    TRAIN_PATH,
    shuffle=True,
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation")

tumor_class_cp_callback = tf.keras.callbacks.ModelCheckpoint(
    CHECK_POINT_PATH.format("tumor_class_6_data_aug-{epoch:02d}-{val_loss:.2f}"),
    monitor='val_acc',
    period=1,
    save_best_only=False,
    verbose=1)

train_data_generator.class_indices

Found 2400 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


{'Benign': 0, 'Malignant': 1}

In [0]:
densenet_tumor_class.fit_generator(
    generator=train_data_generator,
    validation_data=validation_data_generator,
    epochs=EPOCHS,
    callbacks = [tumor_class_cp_callback],
    workers=4)

# Tumor class test

In [28]:
TEST_PATH = "../content/drive/My Drive/BreakHis/Dataset/test/Tumor Class"
CHECK_POINT_PATH = "../content/drive/My Drive/TCC Models/Tumor Class/{}.hdf5"

TARGET_SIZE = (175, 115)
BATCH_SIZE = 64
EPOCHS = 50

test_datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True)

test_data_generator = test_datagen.flow_from_directory(
    TEST_PATH,
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical")

# tumor_class_6_data_aug-15-0.41
#model_file_test = "tumor_class_6_data_aug-05-0.25"
#model_path_test = CHECK_POINT_PATH.format(model_file_test)
#densenet_tumor_class_test = load_model(model_path_test)

Found 1819 images belonging to 2 classes.


In [29]:
import os

MODELS_PATH = r"../content/drive/My Drive/TCC Models/Tumor Class"

models_to_evaluate = []

for r, d, f in os.walk(MODELS_PATH):
  for model in f:
    file = model.split("-")[2]
    val_loss = file.split(".")[0] + "." + file.split(".")[1]
    if float(val_loss) <= 0.60:
      models_to_evaluate.append(model)
      
models_to_evaluate

['tumor_class_data_aug-10-0.60.hdf5',
 'tumor_class_2_data_aug-11-0.58.hdf5',
 'tumor_class_2_data_aug-12-0.46.hdf5',
 'tumor_class_2_data_aug-13-0.56.hdf5',
 'tumor_class_2_data_aug-21-0.42.hdf5',
 'tumor_class_2_data_aug-23-0.55.hdf5',
 'tumor_class_4_data_aug-03-0.59.hdf5',
 'tumor_class_4_data_aug-07-0.58.hdf5',
 'tumor_class_5_data_aug-21-0.44.hdf5',
 'tumor_class_6_data_aug-04-0.18.hdf5',
 'tumor_class_6_data_aug-05-0.25.hdf5',
 'tumor_class_6_data_aug-06-0.32.hdf5',
 'tumor_class_6_data_aug-07-0.51.hdf5',
 'tumor_class_6_data_aug-11-0.46.hdf5',
 'tumor_class_6_data_aug-12-0.42.hdf5',
 'tumor_class_6_data_aug-13-0.44.hdf5',
 'tumor_class_6_data_aug-14-0.56.hdf5',
 'tumor_class_6_data_aug-15-0.41.hdf5',
 'tumor_class_6_data_aug-18-0.51.hdf5',
 'tumor_class_6_data_aug-19-0.56.hdf5']

In [0]:
for model in models_to_evaluate:
  model_path = "../content/drive/My Drive/TCC Models/Tumor Class/{}".format(model)
  densenet_tumor_class_test = load_model(model_path)
  
  test_data_generator.reset()
  score = densenet_tumor_class_test.evaluate_generator(test_data_generator)
  test_data_generator.reset()
  scores = densenet_tumor_class_test.predict_generator(test_data_generator)
  
  print(model)
  print("Loss: ", scores[0], "Accuracy: ", scores[1])
  print(score * 100)

tumor_class_data_aug-10-0.60.hdf5
Loss:  [0.11188126 0.8881187 ] Accuracy:  [0.77968603 0.22031398]
51.837867601164454
tumor_class_2_data_aug-11-0.58.hdf5
Loss:  [0.1019846  0.89801544] Accuracy:  [1. 0.]
49.41921506462426
tumor_class_2_data_aug-12-0.46.hdf5
Loss:  [0.6375286 0.3624713] Accuracy:  [0.5606237  0.43937632]
57.58921442360714
tumor_class_2_data_aug-13-0.56.hdf5
Loss:  [0.66493255 0.33506748] Accuracy:  [0.6853484  0.31465167]
59.19442721481981
